In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [25]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv')
data.head()

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,06:07 AM,07:15 PM


In [4]:
data.shape

(2893, 17)

In [5]:
data.isnull().sum()

day_id                         0
city_id                        0
temperature_celsius            0
condition_text              2414
wind_kph                       0
wind_degree                    0
pressure_mb                    0
precip_mm                      0
humidity                       0
cloud                          0
feels_like_celsius             0
visibility_km                  0
uv_index                       0
gust_kph                       0
air_quality_us-epa-index       0
sunrise                        0
sunset                         0
dtype: int64

In [6]:
# Drop rows with null values
full_data = data.dropna()

In [7]:
full_data.sample(5)

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
2743,D2744,C107,17.0,Mist or Fog,6.8,70,1026.0,0.00,88,0,17.0,8.0,1.0,11.9,1,06:15 AM,07:43 PM
278,D0279,C011,6.0,Mist or Fog,3.6,65,1027.0,0.00,100,0,3.9,9.0,1.0,20.5,1,06:42 AM,07:27 PM
2375,D2376,C092,14.0,Clear and Sunny,6.8,160,1025.0,0.00,23,0,14.0,10.0,1.0,13.7,5,06:14 AM,05:57 PM
869,D0870,C034,24.0,Clear and Sunny,11.2,60,1016.0,0.00,61,0,25.0,10.0,1.0,10.8,2,07:17 AM,08:19 PM
2838,D2839,C110,27.0,Partly Cloudy,19.1,90,1014.0,0.01,70,25,29.8,10.0,6.0,45.6,1,05:39 AM,05:43 PM


In [8]:
full_data.isnull().sum()

day_id                      0
city_id                     0
temperature_celsius         0
condition_text              0
wind_kph                    0
wind_degree                 0
pressure_mb                 0
precip_mm                   0
humidity                    0
cloud                       0
feels_like_celsius          0
visibility_km               0
uv_index                    0
gust_kph                    0
air_quality_us-epa-index    0
sunrise                     0
sunset                      0
dtype: int64

# Data Preprocessing

In [9]:
# Remove unwanted features from the dataset
full_data.drop('day_id', axis=1, inplace=True)
data.drop('day_id', axis=1, inplace=True)

In [10]:
full_data.head()

,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
2,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
6,C001,21.0,Partly Cloudy,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,06:08 AM,07:11 PM
18,C001,19.0,Clear and Sunny,3.6,64,1017.0,0.0,88,0,19.0,10.0,1.0,8.3,3,06:20 AM,06:51 PM
27,C002,19.0,Partly Cloudy,3.6,83,1010.0,0.0,73,25,19.0,10.0,1.0,8.3,1,06:17 AM,07:20 PM


In [11]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 2 to 2888
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   city_id                   479 non-null    object 
 1   temperature_celsius       479 non-null    float64
 2   condition_text            479 non-null    object 
 3   wind_kph                  479 non-null    float64
 4   wind_degree               479 non-null    int64  
 5   pressure_mb               479 non-null    float64
 6   precip_mm                 479 non-null    float64
 7   humidity                  479 non-null    int64  
 8   cloud                     479 non-null    int64  
 9   feels_like_celsius        479 non-null    float64
 10  visibility_km             479 non-null    float64
 11  uv_index                  479 non-null    float64
 12  gust_kph                  479 non-null    float64
 13  air_quality_us-epa-index  479 non-null    int64  
 14  sunrise       

In [12]:
# Conver sunrise and sunset data types into datetime
full_data[['sunrise', 'sunset']] = full_data[['sunrise', 'sunset']].apply(pd.to_datetime, axis=1)
data[['sunrise', 'sunset']] = data[['sunrise', 'sunset']].apply(pd.to_datetime, axis=1)

In [13]:
# Convert city_id data type into numerical
full_data['city_id'] = full_data['city_id'].str.replace('C', '').apply(pd.to_numeric)
data['city_id'] = data['city_id'].str.replace('C', '').apply(pd.to_numeric)

In [14]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 2 to 2888
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   city_id                   479 non-null    int64         
 1   temperature_celsius       479 non-null    float64       
 2   condition_text            479 non-null    object        
 3   wind_kph                  479 non-null    float64       
 4   wind_degree               479 non-null    int64         
 5   pressure_mb               479 non-null    float64       
 6   precip_mm                 479 non-null    float64       
 7   humidity                  479 non-null    int64         
 8   cloud                     479 non-null    int64         
 9   feels_like_celsius        479 non-null    float64       
 10  visibility_km             479 non-null    float64       
 11  uv_index                  479 non-null    float64       
 12  gust_kph                  

In [15]:
# Split sunrise and sunset times into hours and minutes
full_data['sunrise_hour'], full_data['sunrise_minute'] = full_data['sunrise'].dt.hour, full_data['sunrise'].dt.minute
full_data['sunset_hour'], full_data['sunset_minute'] = full_data['sunset'].dt.hour, full_data['sunset'].dt.minute

data['sunrise_hour'], data['sunrise_minute'] = data['sunrise'].dt.hour, data['sunrise'].dt.minute
data['sunset_hour'], data['sunset_minute'] = data['sunset'].dt.hour, data['sunset'].dt.minute

full_data.drop(['sunrise', 'sunset'], axis=1, inplace=True)
data.drop(['sunrise', 'sunset'], axis=1, inplace=True)

In [16]:
full_data.head()

,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_minute,sunset_hour,sunset_minute
2,1,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,6,5,19,18
3,1,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,6,6,19,16
6,1,21.0,Partly Cloudy,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,6,8,19,11
18,1,19.0,Clear and Sunny,3.6,64,1017.0,0.0,88,0,19.0,10.0,1.0,8.3,3,6,20,18,51
27,2,19.0,Partly Cloudy,3.6,83,1010.0,0.0,73,25,19.0,10.0,1.0,8.3,1,6,17,19,20


In [17]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 2 to 2888
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   city_id                   479 non-null    int64  
 1   temperature_celsius       479 non-null    float64
 2   condition_text            479 non-null    object 
 3   wind_kph                  479 non-null    float64
 4   wind_degree               479 non-null    int64  
 5   pressure_mb               479 non-null    float64
 6   precip_mm                 479 non-null    float64
 7   humidity                  479 non-null    int64  
 8   cloud                     479 non-null    int64  
 9   feels_like_celsius        479 non-null    float64
 10  visibility_km             479 non-null    float64
 11  uv_index                  479 non-null    float64
 12  gust_kph                  479 non-null    float64
 13  air_quality_us-epa-index  479 non-null    int64  
 14  sunrise_hour  

In [18]:
# Select dependent and independent features
x = full_data.drop('condition_text', axis=1)
y = full_data['condition_text']

# Test model development using a part of the dataset

In [42]:
# Split dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.head()

,city_id,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_minute,sunset_hour,sunset_minute
252,10,26.5,9.0,152,1006.0,0.0,89,26,30.5,9.0,1.0,14.0,1,5,45,18,3
1809,71,35.0,11.2,220,1010.0,0.0,39,50,37.1,10.0,9.0,12.6,1,6,40,18,54
1980,77,13.0,3.6,10,1021.0,0.0,94,0,12.7,10.0,1.0,12.2,1,5,49,19,22
2565,99,18.0,6.8,110,1014.0,0.0,64,0,18.0,7.0,1.0,2.9,2,6,7,18,32
1449,57,12.0,3.6,10,1022.0,0.0,94,25,12.5,10.0,1.0,6.5,1,5,46,17,45


In [43]:
y_train.head()

252     Light Rain with Thunder
1809    Light Rain with Thunder
1980            Clear and Sunny
2565            Clear and Sunny
1449              Partly Cloudy
Name: condition_text, dtype: object

In [44]:
# Oversampling using SMOTE
test_smote = SMOTE()
x_smote_test, y_smote_test = test_smote.fit_resample(x_train, y_train)

In [45]:
y_smote_test.value_counts()

condition_text
Light Rain with Thunder    96
Clear and Sunny            96
Partly Cloudy              96
Light Precipitation        96
Cloudy and Overcast        96
Rain Showers               96
Moderate to Heavy Rain     96
Thunderstorms              96
Mist or Fog                96
Name: count, dtype: int64

In [46]:
# Model training and get the accuracy
test_model = RandomForestClassifier()
test_model.fit(x_smote_test, y_smote_test)

accuracy_score(y_test, test_model.predict(x_test)) * 100

72.91666666666666

# Model development using the whole dataset

In [47]:
x.head()

,city_id,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_minute,sunset_hour,sunset_minute
2,1,20.0,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,6,5,19,18
3,1,17.0,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,6,6,19,16
6,1,21.0,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,6,8,19,11
18,1,19.0,3.6,64,1017.0,0.0,88,0,19.0,10.0,1.0,8.3,3,6,20,18,51
27,2,19.0,3.6,83,1010.0,0.0,73,25,19.0,10.0,1.0,8.3,1,6,17,19,20


In [48]:
y.value_counts()

condition_text
Clear and Sunny            122
Partly Cloudy              122
Light Precipitation         68
Cloudy and Overcast         53
Mist or Fog                 50
Rain Showers                21
Light Rain with Thunder     16
Thunderstorms               15
Moderate to Heavy Rain      12
Name: count, dtype: int64

In [49]:
# Oversampling using SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)

In [50]:
y_smote.value_counts()

condition_text
Light Rain with Thunder    122
Clear and Sunny            122
Partly Cloudy              122
Light Precipitation        122
Cloudy and Overcast        122
Mist or Fog                122
Rain Showers               122
Moderate to Heavy Rain     122
Thunderstorms              122
Name: count, dtype: int64

In [51]:
# Model training
model = RandomForestClassifier()
model.fit(x_smote, y_smote)

RandomForestClassifier()

# Get predictions

In [52]:
# Get all independent features from the whole dataset
features = data.drop('condition_text', axis=1)
features.head()

,city_id,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_minute,sunset_hour,sunset_minute
0,1,27.0,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,6,4,19,19
1,1,22.0,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,6,5,19,18
2,1,20.0,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,6,5,19,18
3,1,17.0,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,6,6,19,16
4,1,18.0,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,6,7,19,15


In [53]:
features.shape

(2893, 17)

In [54]:
# Get prediction using the model
predictions = model.predict(features)
predictions

array(['Partly Cloudy', 'Partly Cloudy', 'Light Rain with Thunder', ...,
       'Clear and Sunny', 'Clear and Sunny', 'Clear and Sunny'],
      dtype=object)

# Write predictions in the output file

In [55]:
# Read sample submission file
output = pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv')
output.head()

,day_id,condition_text
0,D0001,NaN
1,D0002,NaN
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,NaN


In [56]:
# Remove condition_text from sample submission dataframe
output.drop('condition_text', axis=1, inplace=True)
output.head()

,day_id
0,D0001
1,D0002
2,D0003
3,D0004
4,D0005


In [57]:
# Convert predictions array into dataframe and concatinate it with sample submission
predictions = pd.DataFrame(predictions, columns=['condition_text'])
output = pd.concat([output, predictions], axis=1)
output.head()

,day_id,condition_text
0,D0001,Partly Cloudy
1,D0002,Partly Cloudy
2,D0003,Light Rain with Thunder
3,D0004,Clear and Sunny
4,D0005,Clear and Sunny


In [58]:
# Save submission file
output.to_csv('/kaggle/working/submission.csv', index=False)